</div>
<div align="center">
  <img src="img/evolnode.png" width="800" alt="Fourier reconstruction convergence">
  <p><em> Evolution Node free you from coding and debugging, let LLM evolve your code for you.</em></p>
</div>
<div align="left">
<p><em>Bored of manual coding, a function? EvolNode let LLM automate function design and guide the evolution of it with genetic algorithm. A node here takes a task, input, and output, it uses either code or another LLM to complete the task, ensuring aligned input and output value types and ,.names.Fitness evaluation is done by running the function with a few specified test cases, the more diverse the test case, the better the evolution.
</em></p>
</div>

In [3]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
# mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
# node = EvolNode(mp, None, None)
# input_dict = {"n": 10}
# reasoning, code = node.evolve([input_dict], "i1", replace=True) # Evolution with guaranteed structural fitness
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 
node = EvolNode(mp, None, None)
input_dict = {"name": "Dilireba"}
reasoning, code = node.evolve([input_dict], "i1", replace=True)
node(input_dict)

KeyboardInterrupt: 


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [ ]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)


MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [ ]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [ ]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### RepoRAG
* Smooth Dependency Animation

In [13]:
from tools.repo import *

# (I). Pick top growing repo
# fastest_repos = get_fastest_growing_repos(days_ago=14, top_n = 50, print=False)
# repo_url = fastest_repos[0]['html_url']

# (I). Build a GIF of file-dependency of a git repo
repo_url = "https://github.com/xjdr-alt/entropix.git"
temp_repo = "entropix_repo"

# file_graph = create_gif_from_repo(repo_url, temp_repo, frame_count=60, fps = 10, output_name="entropix_repo_evolution") # File-Level Dependency Graph (not that useful?)
file_dag = build_file_level_dag(repo_url, temp_repo)
# visualize_dag(file_dag) # Slot this image into LLM --> pick a relevant file to zoom in on

# (II). Build function-level dependency graph of a module within the repo


Repo already cloned, skipping cloning...


In [14]:
# Dependency Graph collects all the 'components' required to build the current file (i.e. all the imports) -- useful for minimal implementation and bottom-up understanding

# repo DAG (files..) --> file DAG (functions..) --> function DAG (sub-functions ...) --> sub-function code-snippet

file_dag = build_file_level_dag(repo_url, temp_repo) # repo-level dependency graph (DAG) over all files
# visualize_dag(file_dag) # Slot this image into LLM --> pick a relevant file to zoom in on

python_files = get_python_files(temp_repo) 
python_file = python_files[3] # Makeshift approch to get a 'selected' python file

file_dag = build_cross_file_dag(temp_repo, python_file)
# visualize_dag(file_dag) # Slot this image into LLM --> pick a relevant module to zoom in on 

module_names = get_modules_from_file_dag(file_dag)
module_name = module_names[0] # Makeshift approach to select module from python file

module_dag = extract_subgraph_dag(file_dag, module_name, depth=6)
visualize_dag(module_dag) # Less informative already ...

# function-level depedency graph (DAG) for every file in the repo


Repo already cloned, skipping cloning...
  Number of objects in the subgraph: 4


success: successfully compiled d2_output/dag.d2 to d2_output/dag.png in 259.380125ms


In [5]:
from tools.repo import read_node_content
read_node_content(module_dag['node1'])

"from typing import NamedTuple\n\nimport jax\nimport jax.numpy as jnp\n\n\nclass KVCache(NamedTuple):\n  k: jax.Array\n  v: jax.Array\n\n  @classmethod\n  def new(cls, layers: int, bsz: int, max_seq_len: int, kv_heads: int, head_dim: int) -> 'KVCache':\n    return cls(\n        k=jnp.zeros((layers, bsz, max_seq_len, kv_heads, head_dim), dtype=jnp.bfloat16),\n        v=jnp.zeros((layers, bsz, max_seq_len, kv_heads, head_dim), dtype=jnp.bfloat16)\n    )\n\n  def update(self, xk: jax.Array, xv: jax.Array, layer_idx: int, cur_pos: int, n_rep: int):\n    ck = jax.lax.dynamic_update_slice(self.k, jnp.bfloat16(xk[None, ...]), (layer_idx, 0, cur_pos, 0, 0))\n    cv = jax.lax.dynamic_update_slice(self.v, jnp.bfloat16(xv[None, ...]), (layer_idx, 0, cur_pos, 0, 0))\n    if cur_pos == 0:\n      keys = jnp.repeat(xk, n_rep, axis=2)\n      values = jnp.repeat(xv, n_rep, axis=2)\n    else:\n      keys = jnp.repeat(ck[layer_idx], n_rep, axis=2)\n      values = jnp.repeat(cv[layer_idx], n_rep, axis=2)\

In [1]:
# (II). Build function-level dependency graph of a module within the repo
from tools.repo import *

temp_repo = "hdcnn" # Contains an interesting new DL model architecture's implementation

# def build_dag_from_file(repo_name, file_path):
python_files = get_python_files(temp_repo) # get all python files 

start_file = python_files[0] # pick the first one for debugging purpose 

dag = build_cross_file_dag(temp_repo, start_file)

name_map = {dag[k]["name"]: k for k in dag} # Map name to node-id for all nodes in the DAG

pick_object = list(name_map.keys())[0] # pick the first object from 'start_file'

sub_dag = extract_subgraph_dag(dag, name_map[pick_object], depth=6) # Extact depedency graph starting from pick_object

sub_dag = decide_opacity_of_dag(sub_dag, progress=1.0, cap_node_number=30) # limit number of nodes for easy understanding

# func_graph = create_evolution_gif(sub_dag, frame_count=60, fps=10) # Sick Animation once again ...
# visualize_dag(sub_dag)

  Number of objects in the subgraph: 15


In [ ]:
from methods.llm import get_claude_response # <-- this is the LLM we will be using for summarization
from tools.repo import bottom_up_summarization, save_dag_as_json

summarized_dag = bottom_up_summarization(sub_dag, get_claude_response)

file_path = save_dag_as_json(summarized_dag, temp_repo)

In [1]:
from tools.repo import load_dag 

file_path = "sandbox/hdcnn_dag.json"
file_dag = load_dag(file_path)

In [2]:
from tools.diagram import visualize_dag, file_to_preprocessed_img

png_file_path = visualize_dag(file_dag, show=False)
img_base64 = file_to_preprocessed_img(png_file_path) # ready for visual prompting with LLM

success: successfully compiled d2_output/dag.d2 to d2_output/dag.png in 546.389749ms


In [3]:
from methods.llm import get_claude_response 
from tools.repo import get_navigate_prompt

json_file_path = file_path
user_question = "Describe the structure of this file"
retrieved_info = ""
dag = file_dag

def strategic_respond(json_file_path: str, 
                      user_question: str,
                      file_dag: dict):
    
    retrieved_info = ""
    prompt = get_navigate_prompt(json_file_path, user_question, retrieved_info, file_dag)
    


    response = get_claude_response(prompt, img=img_base64, img_type="image/png")

Could not load vllm class, check CUDA support and GPU RAM size


In [4]:
# Occam Razor: Code-Interpreter  + Dictionary Manipulation 

# Unify: When there is a code-snippet, compile and go to next step, otherwise directly use the response as answer ....

from tools.interpreter import CodeInterpreter

interpreter = CodeInterpreter() 

code_result, figure, has_code = interpreter(response)

In [5]:
print(code_result)
# figure
# has_code

HDCNNClassifier (class):
  Summary: This code defines a simple neural network classifier called HDCNNClassifier. It consists of two fully connected layers with a ReLU activation and dropout in between. The network takes high-dimensional input vectors and outputs class probabilities for multiple classes.
  Dependencies: node18

HDComputing (class):
  Summary: This code defines a class `HDComputing` that implements core operations for hyperdimensional computing. It includes methods for generating random hypervectors, superposing (combining) multiple vectors, binding (associating) two vectors, and permuting (shifting) a vector. These operations are fundamental in hyperdimensional computing for creating and manipulating high-dimensional representations of data.
  Dependencies: node9, node8, node11, node10

AGNewsDataset (class):
  Summary: This code defines a custom PyTorch Dataset class called AGNewsDataset for processing and encoding text data. It preprocesses text by tokenizing, removin

In [4]:
# BreakDown on Task (When something is hard, we break it down)

# 1. Construct step-wise PNG
#    - transparency changing function
#    - growing strategy (code-based subnodes as importance score)
# 2. Animate into GIF (not perfect due to scale disconnection)


# 3. Agent to understand codebase with repo-DAG